# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

In [80]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

vroom_url = 'http://vroom:3000/optimization/'

vehicleProfile = "driving-car"

# DATI posizioni
geoA = json.load(open('../data/mezzolombardo.geojson'))['features']   #4 points
geoB = json.load(open('../data/mezzocorona.geojson'))['features']     #6 points
geoC = json.load(open('../data/piana_rotaliana.geojson'))['features'] #6 points distanti

pointsA = [ p['geometry']['coordinates'] for p in geoA ]
pointsB = [ p['geometry']['coordinates'] for p in geoB ]
pointsC = [ p['geometry']['coordinates'] for p in geoC ]

drt_example = json.load(open('../data/drt_2_veicoli_2_corse.json')) #from vroom shipments example

for v in drt_example['vehicles']:
    v['profile'] = vehicleProfile

center = [11.10260,46.22007]
zoom = 14

point_start = [11.10249,46.21579] #posizione iniziale veicoli
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

def vehicle2Map(v):
    idv = str(v['id'])    
    loc = v['start']
    cap = str(v['capacity'][0])

    try:
        times = v['time_window']
    except:
        times = []
    
    mText = "vehicle %s <br> capacity %s" % (idv, cap)
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = COLORS[idv]
    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "Prenotazione %s <br>" % s['pickup']['id']
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "orario Partenza <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    mText += "Quantita %s <br>" % s['amount'][0]
    
    mark = folium.Marker(revcc(loc1), tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color=color))
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "Prenotazione %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "orario Arrivo <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    mText += "Quantita %s <br>" % s['amount'][0]
    
    mark = folium.Marker(revcc(loc2), tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color=color))
    mark.add_to(Map)

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple',
    '4': 'green'
}

HOURS1 = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]

HOURS2 = [
    [
        hour2sec('14:00'),
        hour2sec('14:05')
    ],
    [
        hour2sec('15:00'),
        hour2sec('15:05')
    ],
    [
        hour2sec('16:00'),
        hour2sec('16:05')
    ],
    [
        hour2sec('17:00'),
        hour2sec('17:05')
    ],
    [
        hour2sec('18:00'),
        hour2sec('18:05')
    ]
]
# VEICOLI

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [1],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        hour2sec('07:00'),
        hour2sec('13:00')
    ],
    #TODO
    #     'breaks': [              #se ci sono orari precisi di pause e sanitizzazione del veicolo
    #         {
    #           "id": 1,
    #           #"description": "pausa sanitizzazione", #descrizione pausa
    #           "service": 300,   #durata pausa 5 minuti
    #           "time_windows": [ #an array of time_window objects describing valid slots for break start
    #             [
    #               hour2sec('13:05'),
    #               hour2sec('13:55')
    #             ],
    #           ]
    #         }
    #       ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = locShift(point_start, 0.003)

vehicle = {
    'id': vid,
    'capacity': [1],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        hour2sec('14:00'),
        hour2sec('19:00')
    ]
}
vehicles.append(vehicle)

#DEFINITION OF JOBS

job_locs = pointsA + pointsB

jobs = []
for jid, cc in enumerate(job_locs):
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna/ritiro(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    #orario casuali
    timeWindow = random.choice(HOURS1)
    
    # shifting casuale del tempo per ogni job
    #s = random.randrange(-3000,3000)
    #timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    job = {
        'id': jid,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna o aggiunta del tempo DI PULIZIA mezzo
        'time_windows': [ timeWindow ]
    }
    jobs.append(job)

request = {
     'vehicles': vehicles,
     'jobs': [],#jobs,
     'shipments': []
}

ships = []
for ship in drt_example['shipments']:
    #ship['pickup']['time_windows']= [[    #pickup == PARTENZA in orario 
    #ship['pickup']['time_windows']= [[]]
    ship['delivery']['time_windows']= [    #delivery == ARRIVO in orario
        random.choice(HOURS1)
#         [
#         hour2sec('11:00'),
#         hour2sec('11:05')
#         ]
    ]
    ship['delivery']['location'] = pointsC.pop()
    ships.append(ship)

request['shipments'] = ships

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

for j in request['jobs']:
    job2Map(j)

for s in request['shipments']:
    print(s)
    ship2Map(s)

Map

{'amount': [1], 'pickup': {'id': 3, 'location': [11.122541427612305, 46.20986352455638]}, 'delivery': {'id': 3, 'location': [11.087179183959961, 46.214169599073806], 'time_windows': [[1617184800, 1617185100]]}}
{'amount': [1], 'pickup': {'id': 4, 'location': [11.123356819152832, 46.21419929496722]}, 'delivery': {'id': 4, 'location': [11.094388961791992, 46.22753112890025], 'time_windows': [[1617192000, 1617192300]]}}


In [63]:
request

{'vehicles': [{'id': 1,
   'capacity': [1],
   'profile': 'driving-car',
   'start': [11.106489999999999, 46.219789999999996],
   'end': [11.106489999999999, 46.219789999999996],
   'time_window': [1617174000, 1617195600]},
  {'id': 2,
   'capacity': [1],
   'profile': 'driving-car',
   'start': [11.10549, 46.21879],
   'end': [11.10549, 46.21879],
   'time_window': [1617199200, 1617217200]}],
 'jobs': [],
 'shipments': [{'amount': [1],
   'pickup': {'id': 3, 'location': [11.122541427612305, 46.20986352455638]},
   'delivery': {'id': 3,
    'location': [11.087179183959961, 46.214169599073806],
    'time_windows': [[1617192000, 1617192300]]}},
  {'amount': [1],
   'pickup': {'id': 4, 'location': [11.123356819152832, 46.21419929496722]},
   'delivery': {'id': 4,
    'location': [11.094388961791992, 46.22753112890025],
    'time_windows': [[1617188400, 1617188700]]}}]}

In [64]:
request['geometry'] = True       #RITORNA GEOMETRIA DELLA SOLUZIONE

resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
solution

{'code': 0,
 'summary': {'cost': 1406,
  'unassigned': 0,
  'delivery': [2],
  'amount': [2],
  'pickup': [2],
  'service': 0,
  'duration': 1406,
  'waiting_time': 2585,
  'priority': 0,
  'distance': 14630,
  'computing_times': {'loading': 11, 'solving': 1, 'routing': 21}},
 'unassigned': [],
 'routes': [{'vehicle': 1,
   'cost': 1406,
   'delivery': [2],
   'amount': [2],
   'pickup': [2],
   'service': 0,
   'duration': 1406,
   'waiting_time': 2585,
   'priority': 0,
   'distance': 14630,
   'steps': [{'type': 'start',
     'location': [11.10649, 46.21979],
     'load': [0],
     'arrival': 1617188243,
     'duration': 0,
     'distance': 0},
    {'type': 'pickup',
     'location': [11.123356819152832, 46.21419929496722],
     'id': 4,
     'service': 0,
     'waiting_time': 0,
     'job': 4,
     'load': [1],
     'arrival': 1617188413,
     'duration': 170,
     'distance': 1999},
    {'type': 'delivery',
     'location': [11.094388961791992, 46.22753112890025],
     'id': 4,
  

In [1]:
#COLORS[ str(random.randrange(1,4)) ]

NameError: name 'COLORS' is not defined

In [65]:
if len(solution['routes']) > 0:
    for route in solution['routes']:
        
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        
        rcolor = COLORS[str(route['vehicle'])]
        #rcolor = COLORS[ str(random.randrange(1,4)) ]
        
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)
        
        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)
        
        i=0
        for step in route['steps']:
            i += 1
            if hasattr(step, 'location'):
                loc = step['location']
                tit = str(i)+" "+step['type']+", arrivo: %s " % sec2hour(route['arrival'])
                
                mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, permanent=True), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
                mark.add_to(Map)
            
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))

Map

Vehicle1 blue distance: 15km delivered quantity: 2 pickup quantity: 2 , durata: 23min 

Total distance: 15km Not Delivered: 0


In [79]:
print(json.dumps(solution, indent=2))

{
  "code": 0,
  "summary": {
    "cost": 1406,
    "unassigned": 0,
    "delivery": [
      2
    ],
    "amount": [
      2
    ],
    "pickup": [
      2
    ],
    "service": 0,
    "duration": 1406,
    "waiting_time": 2585,
    "priority": 0,
    "distance": 14630,
    "computing_times": {
      "loading": 11,
      "solving": 1,
      "routing": 21
    }
  },
  "unassigned": [],
  "routes": [
    {
      "vehicle": 1,
      "cost": 1406,
      "delivery": [
        2
      ],
      "amount": [
        2
      ],
      "pickup": [
        2
      ],
      "service": 0,
      "duration": 1406,
      "waiting_time": 2585,
      "priority": 0,
      "distance": 14630,
      "steps": [
        {
          "type": "start",
          "location": [
            11.10649,
            46.21979
          ],
          "load": [
            0
          ],
          "arrival": 1617188243,
          "duration": 0,
          "distance": 0
        },
        {
          "type": "pickup",
        